In [3]:
import json
path = "wiki-18.jsonl/data00/jiajie_jin/flashrag_indexes/wiki_dpr_100w/wiki_dump.jsonl"

import json

def take_first_n(path, n):
    decoder = json.JSONDecoder()
    items, buf, count = [], '', 0

    with open(path, 'r', encoding='utf-8') as f:
        # read until we have n items or hit EOF
        while count < n:
            chunk = f.read(4096)
            if not chunk:
                break
            buf += chunk

            while count < n:
                buf = buf.lstrip()
                if buf.startswith('['):        # skip opening bracket once
                    buf = buf[1:]
                    buf = buf.lstrip()
                if not buf:
                    break
                if buf[0] == ']':              # reached end of array
                    return items

                try:
                    obj, idx = decoder.raw_decode(buf)
                except json.JSONDecodeError:
                    # need more data
                    break

                items.append(obj)
                count += 1
                buf = buf[idx:].lstrip()
                if buf.startswith(','):
                    buf = buf[1:]              # consume comma before next item

        return items

first = take_first_n(path, 10000)


for x in range(8,11):
    print(x)
    print(len(first[x]["contents"]))
    print(first[x])

8
561
{'id': '8', 'contents': '"Ao Oni (film)"\n(for Takeshi cannot see Shun), and why Anna said Mika will not call him, because he is already dead and cannot be seen by everyone, except Anna. He then flashbacks to the time Takuro bullied him in the riverbank. Takuro hit Shun in the head, killing him. It is then revealed that Takuro went to the empty house to hide Shun\'s body. Takuro then explained that Shun used Takuro\'s name in the game and got mad. Shun is terrified with the truth and finally disappears as Anna apologizes to him. Takuro plans on killing Anna since she knows too much,'}
9
631
{'id': '9', 'contents': '"4th Airborne Corps (Soviet Union)"\n4th Airborne Corps (Soviet Union) The 4th Airborne Corps was an airborne corps of the Red Army in World War II. It fought in the Vyazma airborne operation, an unsuccessful landing during the Rzhev-Vyazma Offensive. The corps was formed in the spring of 1941 in the Western Special Military District from the personnel of the 214th Air

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # Max characters in a chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    length_function=len,  # How to measure chunk length (default for character-based)
    separators=[" ", ""]  # Priority of split characters
)

split = text_splitter.split_text(first[0]["contents"])
print(split[0])

"Evan Morris"
Evan Morris Evan L. Morris (January 26, 1977 – July 9, 2015) was a lobbyist for Genentech and its parent corporation Roche in Washington, D.C. He began his career in Washington as an
